This is the version to run

In [1]:
print("BEGIN DOWNLOAD")
!pip install torch --upgrade
print("DOWNLOADED PYTORCH")
!pip install torchvision --upgrade
print("DOWNLOADED PYTORCH-VISION")

BEGIN DOWNLOAD
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 902.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
print("BEGIN IMPORTING THE LIBRARIES")
import os
import csv
import xml.etree.ElementTree as ET
import json
import torchvision as tv
import torch as t
import numpy as np
import pandas as pd
from torchvision.ops import box_iou
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import ssl
import gc
from PIL import Image
import shutil
import cv2
from torchvision.transforms import functional as F
import psutil
import warnings
warnings.filterwarnings('ignore') 
print("IMPORTED LIBRARIES")
ssl._create_default_https_context = ssl._create_unverified_context
if not os.path.exists('dataset'):
    os.mkdir('dataset')
    os.mkdir('train')
    os.mkdir('test')
    os.mkdir('dataset/drafter_00')
    print("CREATED FOLDERS")

BEGIN IMPORTING THE LIBRARIES
IMPORTED LIBRARIES
CREATED FOLDERS


In [3]:
!cp -r /kaggle/input/cghd1152/* /kaggle/working/dataset

# Preprocessing

In [4]:
print("FILES MOVED")
def read_labels(classes_json: str):
    with open(classes_json, "r", encoding="utf-8") as f:
        label_map = json.load(f)
    return label_map


def xml_to_csv(xml_file, csv_file, label_map):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    with open(csv_file, "w", newline="", encoding="utf-8") as csv_file:
        csv_writer = csv.writer(csv_file)
        headers = [
            "Label",
            "object",
            "xmin",
            "ymin",
            "xmax",
            "ymax",
            "rotation",
            "text",
        ]
        csv_writer.writerow(headers)
        for obj in root.findall(".//object"):
            object_name = obj.find("name").text
            label = label_map.get(object_name, "unknown")
            bndbox = obj.find("bndbox")
            xmin = bndbox.find("xmin").text
            ymin = bndbox.find("ymin").text
            xmax = bndbox.find("xmax").text
            ymax = bndbox.find("ymax").text
            rotation_element = bndbox.find("rotation")
            rotation = rotation_element.text if rotation_element is not None else ""
            text_element = obj.find("text")
            text = text_element.text if text_element is not None else ""
            row = [label, object_name, xmin, ymin, xmax, ymax, rotation, text]
            csv_writer.writerow(row)


def process_drafter_folders(label_map_file):
    label_map = read_labels(label_map_file)
    drafter_folders = [f"/kaggle/working/dataset/drafter_{i}" for i in range(26)]
    for drafter_folder in drafter_folders:
        annotations_folder = os.path.join(drafter_folder, "annotations")
        csv_output_folder = os.path.join(drafter_folder, "csvs")
        os.makedirs(csv_output_folder, exist_ok=True)
        for xml_file in os.listdir(annotations_folder):
            if xml_file.endswith(".xml"):
                xml_file_path = os.path.join(annotations_folder, xml_file)
                csv_file_path = os.path.join(
                    csv_output_folder, f"{os.path.splitext(xml_file)[0]}.csv"
                )

                xml_to_csv(xml_file_path, csv_file_path, label_map)
print("BEGIN CREATING CSVS")
process_drafter_folders("/kaggle/working/dataset/classes.json")

FILES MOVED
BEGIN CREATING CSVS


In [5]:
print("CSV FILES CREATED")

CSV FILES CREATED


In [6]:
print("BEGIN TRAIN-TEST SPLITTING")
for i in range(1, 25):
    img_folder = f"/kaggle/working/dataset/drafter_{i}/images/"
    csv_folder = f"/kaggle/working/dataset/drafter_{i}/csvs/"

    for file in os.listdir(img_folder):
        if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
            shutil.move(img_folder + file, "train/" + file.split(".")[0] + ".jpg")
            shutil.move(
                csv_folder + file.split(".")[0] + ".csv",
                "train/" + file.split(".")[0] + ".csv",
            )


for file in os.listdir("/kaggle/working/dataset/drafter_0/images/"):
    image = cv2.imread("/kaggle/working/dataset/drafter_0/images/" + file)
    cv2.imwrite(
        "/kaggle/working/dataset/drafter_00/" + file.split(".")[0] + ".jpg",
        image,
        [int(cv2.IMWRITE_JPEG_QUALITY), 100],
    )

img_folder = "/kaggle/working/dataset/drafter_00/"
csv_folder = "/kaggle/working/dataset/drafter_0/csvs/"

for file in os.listdir(img_folder):
    if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
        shutil.move(
            img_folder + file,
            "train/" + file.split(".")[0] + ".jpg",
        )
        shutil.move(
            csv_folder + file.split(".")[0] + ".csv",
            "train/" + file.split(".")[0] + ".csv",
    )
print("TRAINING FOLDER FILLED")
for i in range(25, 26):
    img_folder = f"/kaggle/working/dataset/drafter_{i}/images/"
    csv_folder = f"/kaggle/working/dataset/drafter_{i}/csvs/"

    for file in os.listdir(img_folder):
        if file.split('.')[0] + '.csv' in os.listdir(csv_folder):
            shutil.move(img_folder + file, "test/" + file.split(".")[0] + ".jpg")
            shutil.move(
                csv_folder + file.split(".")[0] + ".csv",
                "test/" + file.split(".")[0] + ".csv",
            )
print("TEST FOLDER FILLED")

BEGIN TRAIN-TEST SPLITTING
TRAINING FOLDER FILLED
TEST FOLDER FILLED


In [7]:
print("NUMBER OF TRAINING IMAGES:",len(os.listdir('train')))

NUMBER OF TRAINING IMAGES: 4902


# Model Training

In [8]:
classes = set()
boxes = {}
imgs = {}


directory = f'/kaggle/working/train/'
for file in os.listdir(directory):
        if file.endswith(".csv"):
            df = pd.read_csv(directory + file)
            classes.update(df['object'].tolist())

label_mapping = {label: i for i, label in enumerate(classes, start = 1)}

num_classes = len(classes) + 1
print("PRINT CLASSES TO NUMBERS:", label_mapping)
gc.collect()

PRINT CLASSES TO NUMBERS: {'diode': 1, 'transistor.bjt': 2, 'nor': 3, 'operational_amplifier': 4, 'speaker': 5, 'integrated_circuit': 6, 'voltage.battery': 7, 'voltage.ac': 8, 'resistor': 9, 'optical': 10, 'crystal': 11, 'transistor.photo': 12, 'capacitor.polarized': 13, 'relay': 14, 'crossover': 15, 'switch': 16, 'microphone': 17, 'probe': 18, 'terminal': 19, 'nand': 20, 'mechanical': 21, 'xor': 22, 'vss': 23, 'probe.current': 24, 'diode.zener': 25, 'transistor.fet': 26, 'and': 27, 'text': 28, 'capacitor.adjustable': 29, 'triac': 30, 'capacitor.unpolarized': 31, 'resistor.photo': 32, 'gnd': 33, 'diode.light_emitting': 34, 'voltage.dc': 35, 'integrated_circuit.ne555': 36, 'operational_amplifier.schmitt_trigger': 37, 'magnetic': 38, 'resistor.adjustable': 39, 'not': 40, 'integrated_circuit.voltage_regulator': 41, 'lamp': 42, 'varistor': 43, 'socket': 44, 'unknown': 45, 'thyristor': 46, 'junction': 47, 'optocoupler': 48, 'probe.voltage': 49, 'or': 50, 'antenna': 51, 'transformer': 52, 'i

0

In [9]:
def collate_fn(batch):
    images, targets = zip(*batch)
    images = torch.stack(images)
    return images, targets

class CircuitDataset(Dataset):
    def __init__(self, folder, target_size=(1024, 1024)):
        self.folder = folder
        self.theight, self.twidth = target_size
        self.names = self.indexes()
    
    def indexes(self):
        names = []
        for filename in os.listdir(self.folder):
            if filename.endswith('.csv'):
                name = filename.split('.')[0]
                names.append(name)
        return names
    
    def __len__(self):
        count = 0
        for path in os.listdir(self.folder):
            if path.endswith('.csv'):
                count += 1
        return count

    def __getitem__(self, idx):
        image_path = os.path.join(self.folder, self.names[idx] + '.jpg')
        boxes_path = os.path.join(self.folder, self.names[idx] + '.csv')
        df = pd.read_csv(boxes_path)
        temp = df[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        temp2 = [label_mapping[obj] for obj in df['object'].tolist()]
        boxes = {'boxes': t.from_numpy(temp), 'labels': t.tensor(temp2)}
        del temp, temp2
        gc.collect()
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
        og_dims = torch.tensor([image.width, image.height, image.width, image.height])
        image = F.resize(image, (self.theight, self.twidth))

        scaled_boxes = (boxes['boxes'] / og_dims) * torch.tensor([self.twidth, self.theight, self.twidth, self.theight])
        valid_boxes, valid_indices = self.validate_boxes(scaled_boxes)

        image = F.to_tensor(image)
        boxes['boxes'] = valid_boxes.to(torch.float64)
        boxes['labels'] = torch.tensor([boxes['labels'][i] for i in valid_indices], dtype=torch.int64)
        
        del scaled_boxes
        gc.collect()
        return image, boxes

    def validate_boxes(self, boxes):
        widths = boxes[:, 2] - boxes[:, 0]
        heights = boxes[:, 3] - boxes[:, 1]
        valid_indices = (widths > 0) & (heights > 0)
        return boxes[valid_indices], valid_indices.nonzero(as_tuple=True)[0]

train_folder = f'/kaggle/working/train'
test_folder = f'/kaggle/working/test'
train_dataset = CircuitDataset(train_folder)
print("CREATED TRAIN DATASET")
test_dataset = CircuitDataset(test_folder)
print("CREATED TEST DATASET")

train_data_loader = DataLoader(train_dataset, batch_size = 6, shuffle = True, collate_fn = collate_fn, pin_memory = True)
test_data_loader = DataLoader(test_dataset, batch_size = 6, shuffle = True, collate_fn = collate_fn, pin_memory = True)
model = models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

criterion = nn.SmoothL1Loss()

optimizer = optim.Adam(model.parameters(), lr=1e-5, amsgrad = True)
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.to(device)
else:
    device = torch.device('cpu')
    model.to(device)

CREATED TRAIN DATASET
CREATED TEST DATASET


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:02<00:00, 82.3MB/s]


In [10]:
def print_gpu_memory():
    if torch.cuda.is_available():
        print("GPU Memory:")
        for i in range(torch.cuda.device_count()):
            t = torch.cuda.get_device_properties(i).total_memory
            r = torch.cuda.memory_reserved(i)
            a = torch.cuda.memory_allocated(i)
            f = r-a 
            print(f"  GPU {i}: Total = {t/(1024**3):.2f} GB, Reserved = {r/(1024**3):.2f} GB, Allocated = {a/(1024**3):.2f} GB, Free (within reserved) = {f/(1024**3):.2f} GB")
    else:
        print("No CUDA-capable device is detected")


def print_system_memory():
    memory = psutil.virtual_memory()
    total = memory.total / (1024**3)
    available = memory.available / (1024**3)
    print(f"System Memory: Total = {total:.2f} GB, Available = {available:.2f} GB")

In [11]:
print("TRAINING MODEL")
num_epochs = 25
model.train()
for epoch in range(num_epochs):
    running_loss = None
    for images, targets in train_data_loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.sum().backward()
        optimizer.step()
        if running_loss is None:
            running_loss = losses.sum().cpu().detach().numpy()
        else: 
            running_loss += losses.sum().cpu().detach().numpy()
        del losses, loss_dict, images, targets
        torch.cuda.empty_cache() ##This is to make sure it does not crash
        gc.collect()
    print_gpu_memory()
    print_system_memory()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_data_loader)}')
print("MODEL TRAINED")
    
torch.save(model.state_dict(), 'faster_rcnn_model.pth')

TRAINING MODEL
GPU Memory:
  GPU 0: Total = 15.89 GB, Reserved = 3.42 GB, Allocated = 0.84 GB, Free (within reserved) = 2.58 GB
System Memory: Total = 31.36 GB, Available = 29.03 GB
Epoch [1/25], Loss: 2.4225558538792895
GPU Memory:
  GPU 0: Total = 15.89 GB, Reserved = 3.45 GB, Allocated = 0.85 GB, Free (within reserved) = 2.60 GB
System Memory: Total = 31.36 GB, Available = 28.99 GB
Epoch [2/25], Loss: 1.4264000684936897
GPU Memory:
  GPU 0: Total = 15.89 GB, Reserved = 3.13 GB, Allocated = 0.85 GB, Free (within reserved) = 2.29 GB
System Memory: Total = 31.36 GB, Available = 29.00 GB
Epoch [3/25], Loss: 1.235940048644647
GPU Memory:
  GPU 0: Total = 15.89 GB, Reserved = 2.65 GB, Allocated = 0.85 GB, Free (within reserved) = 1.80 GB
System Memory: Total = 31.36 GB, Available = 29.03 GB
Epoch [4/25], Loss: 1.137596041943362
GPU Memory:
  GPU 0: Total = 15.89 GB, Reserved = 2.71 GB, Allocated = 0.85 GB, Free (within reserved) = 1.86 GB
System Memory: Total = 31.36 GB, Available = 29.03

In [12]:
print("SAVED MODEL")

SAVED MODEL


In [13]:
print("EVALUATING MODEL ON CLASSES")
model.eval()
all_predictions = []
all_targets = []
with torch.no_grad():
    for images, targets in test_data_loader:
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        predictions = model(images)
        all_predictions.extend(predictions)
        all_targets.extend(targets)
print("PREDICTION COMPLETED, STARTING EVALUATION")

EVALUATING MODEL ON CLASSES
PREDICTION COMPLETED, STARTING EVALUATION


In [14]:
def evaluate_model(all_predictions, all_targets, label_mapping, iou_threshold=0.5):
    index_to_class = {v: k for k, v in label_mapping.items()}
    class_metrics = {index_to_class[i]: {'TP': 0, 'FP': 0, 'FN': 0, 'Avg_IoU': 0, 'Total_IoU': 0, 'Matched_IoU_Count': 0} for i in label_mapping.values()}

    for predictions, targets in zip(all_predictions, all_targets):
        pred_boxes = torch.tensor(predictions['boxes']).to(device)
        pred_labels = torch.tensor(predictions['labels']).to(device)
        target_boxes = torch.tensor(targets['boxes']).to(device)
        target_labels = torch.tensor(targets['labels']).to(device)

        for cls_index, cls_name in index_to_class.items():
            cls_pred_boxes = pred_boxes[pred_labels == cls_index]
            cls_target_boxes = target_boxes[target_labels == cls_index]

            if len(cls_pred_boxes) == 0 and len(cls_target_boxes) == 0:
                continue
            if len(cls_pred_boxes) == 0:
                class_metrics[cls_name]['FN'] += len(cls_target_boxes)
                continue
            if len(cls_target_boxes) == 0:
                class_metrics[cls_name]['FP'] += len(cls_pred_boxes)
                continue

            iou_matrix = box_iou(cls_pred_boxes, cls_target_boxes)
            iou_max, iou_max_idx = iou_matrix.max(dim=1)
            matched_preds = iou_max >= iou_threshold

            class_metrics[cls_name]['Total_IoU'] += iou_max[matched_preds].sum().item()
            class_metrics[cls_name]['Matched_IoU_Count'] += matched_preds.sum().item()

            class_metrics[cls_name]['TP'] += matched_preds.sum().item()
            class_metrics[cls_name]['FP'] += (~matched_preds).sum().item()
            class_metrics[cls_name]['FN'] += len(cls_target_boxes) - matched_preds.sum().item()

    classwise_results = {}
    for cls_name, metrics in class_metrics.items():
        precision = metrics['TP'] / (metrics['TP'] + metrics['FP']) if (metrics['TP'] + metrics['FP']) > 0 else 0
        recall = metrics['TP'] / (metrics['TP'] + metrics['FN']) if (metrics['TP'] + metrics['FN']) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        avg_iou = (metrics['Total_IoU'] / metrics['Matched_IoU_Count']) if metrics['Matched_IoU_Count'] > 0 else 0

        if precision > 0 or recall > 0 or f1_score > 0 or avg_iou > 0:
            classwise_results[cls_name] = {
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score,
                "avg_iou": avg_iou
            }

    df = pd.DataFrame.from_dict(classwise_results, orient='index')
    df.to_csv("class_metrics.csv")
    return classwise_results
print("BEGIN EVALUATION")
results = evaluate_model(all_predictions, all_targets, label_mapping)
print("EVALUATION COMPLETED")
print(results)

BEGIN EVALUATION
EVALUATION COMPLETED
{'diode': {'precision': 0.176, 'recall': 0.9166666666666666, 'f1_score': 0.29530201342281875, 'avg_iou': 0.829614069371218}, 'transistor.bjt': {'precision': 0.7664233576642335, 'recall': 0.9375, 'f1_score': 0.8433734939759037, 'avg_iou': 0.8611452188929767}, 'integrated_circuit': {'precision': 0.057971014492753624, 'recall': 1.0, 'f1_score': 0.1095890410958904, 'avg_iou': 0.9132800909279923}, 'resistor': {'precision': 0.534375, 'recall': 0.7633928571428571, 'f1_score': 0.6286764705882354, 'avg_iou': 0.8803568173500972}, 'capacitor.polarized': {'precision': 0.25396825396825395, 'recall': 1.0, 'f1_score': 0.4050632911392405, 'avg_iou': 0.8669846265382553}, 'crossover': {'precision': 0.7609561752988048, 'recall': 0.09138755980861243, 'f1_score': 0.16317812900469883, 'avg_iou': 0.7098730808134104}, 'switch': {'precision': 0.4583333333333333, 'recall': 0.5729166666666666, 'f1_score': 0.5092592592592592, 'avg_iou': 0.7094274082045323}, 'terminal': {'prec

In [15]:
shutil.rmtree('dataset')
shutil.rmtree('train')
shutil.rmtree('test')
print("NOTEBOOK FINISHED")

NOTEBOOK FINISHED
